In [29]:
from dotenv import load_dotenv

load_dotenv()
import os

In [30]:
# wrapper on top of wikipedia API
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [31]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [32]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://python.langchain.com/docs/modules/agents/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500).split_documents(docs)
embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
# Create a FAISS vector store from the documents
vectordb = FAISS.from_documents(documents, embedding)
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000221FA6CE150>, search_kwargs={})

In [33]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,name='AgentSearchTool', 
                      description="LangChain Agent Search Tool. Use this tool to search for information related to LangChain agents. ")

In [34]:
retriever_tool.name

'AgentSearchTool'

In [35]:
# Arxiv tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=500)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [36]:
tools = [wiki, retriever_tool, arxiv]

In [37]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Admin\\Desktop\\LANGCHAIN\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=500)),
 Tool(name='AgentSearchTool', description='LangChain Agent Search Tool. Use this tool to search for information related to LangChain agents. ', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000221FD62EFC0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000221FA6CE150>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000

In [40]:
### Agents
from langchain_groq import ChatGroq
from langchain.agents import initialize_agent, AgentType

llm = ChatGroq(
    model="groq/llama-3.1-70b-instruct",
    temperature=0.1,
)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    prompt="""You are a helpful assistant that can answer questions about LangChain agents,
    Wikipedia, and Arxiv. Use the tools provided to find information."""
)


C:\Users\Admin\AppData\Local\Temp\ipykernel_9536\4271997069.py:10: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [ ]:
### Agent executer
from langchain.agents import AgentExecutor
